<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

### Datos
Utilizaremos como dataset canciones de bandas de habla inglés.

In [2]:
# Descargar la carpeta de dataset
import os
import platform
if os.access('./songs_dataset', os.F_OK) is False:
    if os.access('songs_dataset.zip', os.F_OK) is False:
        if platform.system() == 'Windows':
            !curl https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip -o songs_dataset.zip
        else:
            !wget songs_dataset.zip https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip
    !unzip -q songs_dataset.zip   
else:
    print("El dataset ya se encuentra descargado")

--2022-03-31 22:03:39--  http://songs_dataset.zip/
Resolving songs_dataset.zip (songs_dataset.zip)... failed: Name or service not known.
wget: unable to resolve host address ‘songs_dataset.zip’
--2022-03-31 22:03:39--  https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip [following]
--2022-03-31 22:03:40--  https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.

In [3]:
# Posibles bandas
os.listdir("./songs_dataset/")

['bjork.txt',
 'bruce-springsteen.txt',
 'beatles.txt',
 'radiohead.txt',
 'disney.txt',
 'nursery_rhymes.txt',
 'rihanna.txt',
 'eminem.txt',
 'dickinson.txt',
 'dr-seuss.txt',
 'notorious_big.txt',
 'joni-mitchell.txt',
 'lil-wayne.txt',
 'lady-gaga.txt',
 'nickelback.txt',
 'r-kelly.txt',
 'adele.txt',
 'janisjoplin.txt',
 'alicia-keys.txt',
 'patti-smith.txt',
 'lorde.txt',
 'kanye-west.txt',
 'amy-winehouse.txt',
 'al-green.txt',
 'cake.txt',
 'ludacris.txt',
 'Lil_Wayne.txt',
 'lin-manuel-miranda.txt',
 'paul-simon.txt',
 'drake.txt',
 'nicki-minaj.txt',
 'bruno-mars.txt',
 'blink-182.txt',
 'johnny-cash.txt',
 'britney-spears.txt',
 'michael-jackson.txt',
 'kanye.txt',
 'Kanye_West.txt',
 'dolly-parton.txt',
 'notorious-big.txt',
 'bob-marley.txt',
 'nirvana.txt',
 'bieber.txt',
 'prince.txt',
 'jimi-hendrix.txt',
 'missy-elliott.txt',
 'bob-dylan.txt',
 'dj-khaled.txt',
 'leonard-cohen.txt']

In [4]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
df = pd.read_csv('songs_dataset/bob-marley.txt', sep='/n', header=None)
df.head()

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


,0
0,"""Don't worry about a thing,"
1,'Cause every little thing gonna be all right.
2,"Singin': ""Don't worry about a thing,"
3,"'Cause every little thing gonna be all right!""..."
4,"Smiled with the risin' sun,"


In [5]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 2218


### 1 - Preprocesamiento

In [6]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [7]:
# Demos un vistazo
sentence_tokens[:2]

[["don't", 'worry', 'about', 'a', 'thing'],
 ["'cause", 'every', 'little', 'thing', 'gonna', 'be', 'all', 'right']]

### 2 - Crear los vectores (word2vec)

In [8]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobracargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [9]:
# Crearmos el modelo generador de vectoeres
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     size=300,       # dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [10]:
# Buildear el vocabularui con los tokens
w2v_model.build_vocab(sentence_tokens)

In [11]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 2218


In [12]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.vocab))

Cantidad de words distintas en el corpus: 532


### 3 - Entrenar el modelo generador

In [13]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=30,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 154141.8125
Loss after epoch 1: 93114.96875
Loss after epoch 2: 92252.84375
Loss after epoch 3: 92657.625
Loss after epoch 4: 92977.25
Loss after epoch 5: 90923.0625
Loss after epoch 6: 84578.6875
Loss after epoch 7: 80709.0
Loss after epoch 8: 76268.0
Loss after epoch 9: 72253.1875
Loss after epoch 10: 70306.5625
Loss after epoch 11: 66671.25
Loss after epoch 12: 61388.625
Loss after epoch 13: 59111.625
Loss after epoch 14: 58743.875
Loss after epoch 15: 58017.0
Loss after epoch 16: 56643.0
Loss after epoch 17: 56101.5
Loss after epoch 18: 55255.125
Loss after epoch 19: 54092.75
Loss after epoch 20: 53609.75
Loss after epoch 21: 53057.75
Loss after epoch 22: 52746.5
Loss after epoch 23: 52785.0
Loss after epoch 24: 52169.875
Loss after epoch 25: 52183.375
Loss after epoch 26: 52237.5
Loss after epoch 27: 51469.125
Loss after epoch 28: 50638.75
Loss after epoch 29: 50958.0


(327015, 538890)

### 4 - Ensayar

In [14]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["one"], topn=10)

[("cryin'", 0.7487215995788574),
 ('ask', 0.6759779453277588),
 ('really', 0.6530627608299255),
 ("i'd", 0.623403787612915),
 ("let's", 0.6201443076133728),
 ('bright', 0.6076955199241638),
 ('drop', 0.6066855192184448),
 ('creation', 0.6044543981552124),
 ("sayin'", 0.6035938858985901),
 ('another', 0.5962762832641602)]

In [15]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["love"], topn=10)

[('bob', -0.07967435568571091),
 ('walk', -0.11529499292373657),
 ('marley', -0.1174994632601738),
 ('fly', -0.13649116456508636),
 ('natty', -0.1559484302997589),
 ("she's", -0.16601809859275818),
 ('long', -0.17485854029655457),
 ('ride', -0.17596027255058289),
 ('cut', -0.1764945685863495),
 ('gone', -0.17849385738372803)]

In [16]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["give"], topn=10)

[('thanks', 0.7406505346298218),
 ('praise', 0.708186149597168),
 ('treat', 0.685573160648346),
 ('alone', 0.6424753069877625),
 ('nobody', 0.6365970373153687),
 ('boat', 0.6190796494483948),
 ('birds', 0.6087950468063354),
 ('satisfied', 0.604256272315979),
 ("pickin'", 0.595650851726532),
 ('blood', 0.5945072174072266)]

In [17]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["music"], topn=5)

[('hit', 0.8743365406990051),
 ("chuckin'", 0.8319815993309021),
 ('brother', 0.7783136367797852),
 ('dance', 0.7374036908149719),
 ('hurt', 0.717072606086731)]

In [18]:
# Ensayar con una palabra que no está en el corpus (en vocab):
w2v_model.wv.most_similar(negative=["diedaa"])

KeyError: ignored

### 5 - Visualizar agrupación de vectores

In [19]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model):
    num_dimensions = 2  

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index2word)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels

In [20]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

x_vals, y_vals, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=x_vals[:MAX_WORDS], y=y_vals[:MAX_WORDS], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,
